**Data Loading and Preparation**

In [1]:
#import pandas library
import pandas as pd

# Load sales and supermarket data
sales_df = pd.read_csv(r"C:\Users\09sra\Downloads\Data Science and Engineering Assignment Datasets\Data Sciene Internship Assignment Datasets\sales.csv")
sm_df = pd.read_csv(r"C:\Users\09sra\Downloads\Data Science and Engineering Assignment Datasets\Data Sciene Internship Assignment Datasets\supermarkets.csv")

In [2]:
# Renaming to reduce ambiguity 
sm_df = sm_df.rename(columns={'supermarket_No': 'supermarket'})

In [3]:
# Joining sales and supermarket data using left-join
merged_df = pd.merge(sales_df, sm_df, on='supermarket', how='left')

**Feature Engineering**

In [14]:
# Aggregating the data into week level for each supermarket
weekly_store_df = merged_df.groupby(['supermarket', 'week'], as_index=False).agg({
    'amount': 'sum',        # Total amount generated each week        
    'units': 'sum',         # No of Units Sold in Total
    'basket': 'nunique',    # Total unique Number of Transcations
    'voucher': 'sum',       # No of Vouchers used
    'province': 'first',    
    'postal-code': 'first'   
})

# Rename column names 
weekly_store_df.rename(columns={
    'amount': 'total_revenue',
    'units': 'total_units',
    'basket': 'num_transactions',
    'voucher': 'total_vouchers'
}, inplace=True)

In [5]:
# Sort by supermarket and week for time-series Analysis 
weekly_store_df = weekly_store_df.sort_values(['supermarket', 'week'])

# Next week's revenue as the target value label 
weekly_store_df['future_revenue'] = weekly_store_df.groupby('supermarket')['total_revenue'].shift(-1)

In [6]:
# Removed rows which has no future revenue generated 
weekly_store_df = weekly_store_df.dropna(subset=['future_revenue'])

In [7]:
# Creating Lagged Features

weekly_store_df['revenue_last_week'] = weekly_store_df.groupby('supermarket')['total_revenue'].shift(1)
weekly_store_df['units_last_week']   = weekly_store_df.groupby('supermarket')['total_units'].shift(1)
weekly_store_df['numtxn_last_week']   = weekly_store_df.groupby('supermarket')['num_transactions'].shift(1)
weekly_store_df['totalvouchers_last_week']   = weekly_store_df.groupby('supermarket')['total_vouchers'].shift(1)



# fill any new NaN from the lagging with 0
weekly_store_df.fillna(0, inplace=True)

**Test and Train data Splitting**

In [8]:
# define a cutoff in between test and train 
train_cutoff = 24
# Splitting data into train(from weeks 1-24) and test(from weeks 25-27)  
train_df = weekly_store_df[weekly_store_df['week'] <= train_cutoff]
test_df  = weekly_store_df[(weekly_store_df['week'] > train_cutoff) & (weekly_store_df['week'] < 28)]


In [9]:
# define selected features and target 
feature_cols = [
    'total_revenue', 'total_units', 'num_transactions', 'total_vouchers',
    'revenue_last_week', 'units_last_week','numtxn_last_week', 'totalvouchers_last_week' 
]

X_train = train_df[feature_cols]
y_train = train_df['future_revenue']

X_test = test_df[feature_cols]
y_test = test_df['future_revenue']

**Traning and Tuning the Model**

In [10]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Define a base Random Forest model
base_model = RandomForestRegressor(random_state=42)

# Set up a hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=5,                # 5-fold cross-validation
    scoring='r2',        # using R^2 as the metric
    n_jobs=-1,           # use all available cores
    verbose=1
)

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Output best hyperparameters and corresponding CV score 
print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation R^2 score:", grid_search.best_score_)

# Use the best estimator from grid search as the final model
model = grid_search.best_estimator_

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2   = r2_score(y_test, y_pred)
print("Test RMSE:", rmse)
print("Test R^2:", r2)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters found: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 300}
Best cross-validation R^2 score: 0.5312792019958545
Test RMSE: 112.64022492426157
Test R^2: 0.26213120559257574


**Forecasting Future Performance**

In [ ]:
# Prediction of the final 4 weeks( weeks 24-27) from the aggregated data
future_weeks_df = weekly_store_df[weekly_store_df['week'] >= 24].copy()
future_weeks_df.sort_values(['supermarket','week'], inplace=True)

# Create lag features the same way you did for training
future_weeks_df['revenue_last_week'] = future_weeks_df.groupby('supermarket')['total_revenue'].shift(1)
future_weeks_df['units_last_week'] = future_weeks_df.groupby('supermarket')['total_units'].shift(1)

# Fill missing values
future_weeks_df.fillna(0, inplace=True)

# Select feature columns for the model
feature_cols = ['total_revenue', 'total_units', 'num_transactions', 'total_vouchers',
    'revenue_last_week', 'units_last_week','numtxn_last_week', 'totalvouchers_last_week' ]
X_future = future_weeks_df[feature_cols]

# Predict next week's revenue
future_weeks_df['predicted_revenue'] = model.predict(X_future)

# Now each row in future_weeks_df has predicted_revenue for next week
future_weeks_df[['supermarket','week','predicted_revenue']].head(20)

,supermarket,week,predicted_revenue
19,1,24,141.536061
20,1,25,251.797437
21,1,26,265.668832
22,1,27,259.323485
43,2,24,221.192839
44,2,25,192.273156
45,2,26,211.403721
46,2,27,260.561531
67,3,24,170.746882
68,3,25,237.041508


In [ ]:
# 
future_weeks_df['is_high_perf'] = False
for t in [25, 26, 27, 28]:
    mask_t = (future_weeks_df['week'] == t)
    mean_pred = future_weeks_df.loc[mask_t, 'predicted_revenue'].mean()
    threshold = 1.25 * mean_pred

    future_weeks_df.loc[mask_t, 'is_high_perf'] = (
        future_weeks_df.loc[mask_t, 'predicted_revenue'] >= threshold
    )

# Creating a consecutive_high column where no supermarket has high performing weeks 
future_weeks_df['consecutive_high'] = False
# Sorting the data by supermarket and week to maintain data in chronological way for each Supermarket
future_weeks_df = future_weeks_df.sort_values(['supermarket','week'])


for sm_id in future_weeks_df['supermarket'].unique():
    sm_mask = (future_weeks_df['supermarket'] == sm_id)
    sm_data = future_weeks_df.loc[sm_mask].copy().sort_values('week')

    is_hp_array = sm_data['is_high_perf'].values  
    consecutive_arr = [False] * len(is_hp_array)
    # Identifying high performing weeks for the supermarket consecutively
    for i in range(1, len(is_hp_array)):
        if is_hp_array[i] and is_hp_array[i-1]:
            consecutive_arr[i] = True

    future_weeks_df.loc[sm_mask, 'consecutive_high'] = consecutive_arr

# Final High Performing Supermarkets
high_perf_stores = future_weeks_df.loc[future_weeks_df['consecutive_high'],'supermarket'].unique()
print("High-performing supermarkets:", high_perf_stores)

High-performing supermarkets: [  5  16  17  23  27  32  33  35  40  54  71  91  92  97 103 106 107 109
 112 126 135 162 164 172 174 181 184 186 190 193 196 202 215 218 227 234
 235 240 241 247 248 259 264 270 271 276 277 280 285 302 307 308 309 316
 321 322 324 326 341 353 357 359 364 366 368 371 373 375 378 380 381 383]
